In [ ]:
pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 40.3 MB/s eta 0:00:00


In [ ]:
pip install pyomo

In [ ]:
%pip install gurobipy

In [ ]:
pip install -U "scipy==1.4.*" "pulp==2.1"

ERROR: Operation cancelled by user


In [ ]:
!pip install ortools
from ortools.linear_solver import pywraplp

In [ ]:
import pandas as pd
import cvxpy as cp
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from mip import *
from pylab import *
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
from pulp import GLPK
from scipy.optimize import linprog
import xlrd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import pyomo.environ as pyo
from pyomo.environ import *

model = ConcreteModel()
model.T = RangeSet(5)    # time periods

i0 = 5.0           # initial inventory
c = 4.6            # setup cost
h_pos = 0.7        # inventory holding cost
h_neg = 1.2        # shortage cost
P = 5.0            # maximum production amount

# demand during period t
d = {1: 5.0, 2:7.0, 3:6.2, 4:3.1, 5:1.7}


# TODO: WRITE CODE FOR THE MODEL HERE

# solve the problem
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
for t in model.T:
    print('Period: {0}, Prod. Amount: {1}'.format(t, value(model.x[t])))

In [ ]:
# Parameters

jenis_bahan_baku = [1,2,3]
periode_waktu_datang = [1,t]
periode_waktu_pesan = [1,t']

curr_workforce = {'s1': 2000, 's2': 1500, 's3': 1000}
demand = {
    (1, 's1'): 1000,
    (1, 's2'): 1400,
    (1, 's3'): 1000,
    (2, 's1'): 500,
    (2, 's2'): 2000,
    (2, 's3'): 1500,
    (3, 's1'): 0,
    (3, 's2'): 2500,
    (3, 's3'): 2000
}
max_overmanning = 150
max_parttime = 50
parttime_cap = 0.50
max_train_unskilled = 200
max_train_semiskilled = 0.25

training_cost = {'s1': 400, 's2': 500}
layoff_cost = {'s1': 200, 's2': 500, 's3': 500}
parttime_cost = {'s1': 500, 's2': 400, 's3': 400}
overmanning_cost = {'s1': 1500, 's2': 2000, 's3': 3000}

# Number of workers required for each shift.
shifts, shiftRequirements = gp.multidict({
  "Mon1":  3,
  "Tue2":  2,
  "Wed3":  4,
  "Thu4":  4,
  "Fri5":  5,
  "Sat6":  6,
  "Sun7":  5,
  "Mon8":  2,
  "Tue9":  2,
  "Wed10": 3,
  "Thu11": 4,
  "Fri12": 6,
  "Sat13": 7,
  "Sun14": 5 })

x = m.addVars(availability, vtype=GRB.BINARY, name="x")
slacks = m.addVars(shifts, name="Slack")

# Constraint: All shifts requirements most be satisfied.

shift_reqmts = m.addConstrs((x.sum('*',s) + slacks[s] == shiftRequirements[s] for s in shifts), name='shiftRequirement')

m.setObjectiveN(totSlack, index=0, priority=2, reltol=0.2, name='TotalSlack')

m.setObjectiveN(maxShift - minShift, index=1, priority=1, name='Fairness'
m.write('workforce.lp')
m.optimize()

status = m.Status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE  or status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is infeasible or unbounded')
    sys.exit(0)
# If the optimization status of the model is not optimal for some other reason, we report that
# situation.
if status != GRB.Status.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))
    sys.exit(0)

# Print total slack and the number of shifts worked for each worker
# The KPIs for this optimization number is the number of extra worked required to satisfy
# demand and the number of shifts that each employed worker is working.
solution = {}
shifts_sol = {}
solution['Total slack required'] = str(totSlack.X)
assignments_all = {}
gant={}

assignments = dict()
for [w, s] in availability:
    if x[w, s].x == 1:
        if w in assignments:
            assignments[w].append(s)
        else:
            assignments[w] = [s]


print(pd.DataFrame.from_records(list(solution.items()), columns=['KPI', 'Value']))
print('-'*50)

for w in workers:
    shifts_sol[w]=totShifts[w].X
    assignments_all[w]=assignments.get(w, [])

print('Shifts')
print(pd.DataFrame.from_records(list(shifts_sol.items()), columns=['Worker', 'Number of shifts']))

y_pos = np.arange(len(shifts_sol.keys()))
plt.bar(y_pos,shifts_sol.values() , align='center')
plt.xticks(y_pos, shifts_sol.keys())
plt.show()

print('-'*50)
for w in assignments_all:
    gant[w] = [w]
    for d in shifts:
        gant[w].append('*' if d in assignments_all[w] else '-')

print('Assigments')
print('Symbols: \'-\': not working, \'*\': working')
pd.set_option('display.width', 1000)
print(pd.DataFrame.from_records(list(gant.values()), columns=['worker']+shifts))

In [ ]:
#0.1 Objective Function: Minimize layoffs
obj1 = layoff.sum()
manpower.setObjective(obj1, GRB.MINIMIZE)

obj2 = quicksum((training_cost[level]*train[year, level, skills[skills.index(level)+1]] if level < 's3' else 0)
                + layoff_cost[level]*layoff[year, level]
                + parttime_cost[level]*part_time[year, level]
                + overmanning_cost[level] * excess[year, level] for year in years for level in skills)

In [ ]:
rows = years.copy()
columns = skills.copy()
hire_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in hire.keys():
    if (abs(hire[year, level].x) > 1e-6):
        hire_plan.loc[year, level] = np.round(hire[year, level].x, 1)
hire_plan

In [ ]:
m = Model()

In [ ]:
m = Model(sense=MAXIMIZE, solver_name=CBC)

In [ ]:
x = m.add_var()

In [ ]:
c = [90, 120, 100, 80, 130]
b = [45, 60, 50]
a = [[10, 15, 12, 9, 13],[20, 15, 25, 15, 10],[15, 20, 20, 15, 10]]
T, I = range(len(b)),range(len(c))

In [ ]:
x = [[m.add_var(name = names[i], var_type=BINARY) for i in I] for t in T]

In [ ]:
m.objective = maximize(xsum(c[i] * x[t][i] for i in I for t in T))

In [ ]:
n = 90
y = [ m.add_var(var_type=BINARY) for i in range(n) ]

In [ ]:
m += x + y <= 10

In [ ]:
m += xsum(w[i]*x[i] for i in range(n)) <= c

In [ ]:
m += xsum(w[i]*x[i] for i in range(n) if i%2 == 0) <= c
m += xsum(w[i]*x[i] for i in range(n) if i%2 == 0) <= c, 'even_sum'

In [ ]:
constraint = m.constr_by_name('even_sum')

In [ ]:
m.objective = xsum(c[i]*x[i] for i in range(n))
m.objective = minimize(xsum(c[i]*x[i] for i in range(n)))
m.objective = maximize(xsum(c[i]*x[i] for i in range(n)))

In [ ]:
m.write('model.lp')

In [ ]:
m.read('model.lp')
print('model has {} vars, {} constraints and {} nzs'.format(m.num_cols, m.num_rows, m.num_nz))

In [ ]:
m.max_gap = 0.05
status = m.optimize(max_seconds=300)
if status == OptimizationStatus.OPTIMAL:
print('optimal solution cost {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
print('sol.cost {} found, best possible: {} '.format(m.objective_value, m.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
print('no feasible solution found, lower bound is: {} '.format(m.objective_bound))
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
print('solution:')
for v in m.vars:
if abs(v.x) > 1e-6: # only printing non-zeros
print('{} : {} '.format(v.name, v.x))

SyntaxError: ignored

In [ ]:
for k in range(model.num_solutions):
print('route {} with length {} '.format(k, model.objective_values[k]))
for (i, j) in product(range(n), range(n)):
if x[i][j].xi(k) >= 0.98:
print('\tarc ({} ,{} )'.format(i,j))

In [ ]:
results = {}
for i in range(3):
 results[i+1] = [m.vars[j].x for j in range(5*i,5*i+5)]

In [ ]:
result = pd.DataFrame(data = results, index = ['A','B','C','D','E'])

In [ ]:
from mip import Model, MAXIMIZE, CBC, INTEGER, OptimizationStatus
>>> model = Model(sense=MAXIMIZE, solver_name=CBC)
>>> x = model.add_var(name='x', var_type=INTEGER, lb=0, ub=10)
>>> y = model.add_var(name='y', var_type=INTEGER, lb=0, ub=10)
>>> model += x + y <= 10
>>> model.objective = x + y
>>> status = model.optimize(max_seconds=2)
>>> status == OptimizationStatus.OPTIMAL
True

In [ ]:
m += x1 + x2 <= 1
m.add_constr( x1 + x2 <= 1 )

In [ ]:
m += xsum(x[i] for i in range(n)) == y, "cons1"
m.add_constr( xsum(x[i] for i in range(n)) == y, "cons1" )

In [ ]:
x = m.add_var()
x = [m.add_var(var_type=BINARY) for i in range(n)]


In [ ]:
check_optimization_results()

In [ ]:
m.objective = 10*x1 + 7*x4

In [ ]:
execution_time_t0 = time.time()

#code blablabla ...

execution_time_t1 = time.time()
print(execution_time_t1-execution_time_t0)

Note: If you want to write your solution to a file, rather than print it to the terminal, you can use the model.write() command. An example implementation is:

manpower.write("manpower-planning-output.sol")